In [3]:
# Core imports
import heyoka as hk
import pykep as pk
import numpy as np
from numpy import linalg as la
import pandas as pd
import math
from numpy.lib.function_base import append
from scipy.integrate import odeint
# Sys imports
import time
# Plot imports
from matplotlib import pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.ticker as mtick
from UROP_aux_func import rv2elorb

In [4]:
# Step 1: Define Initial Conditions (dimensional)
r0 = np.array([0.0, -5888.9727, -3400.0]) #km   (class 'numpy.ndarray')
v0 = np.array([10.691338, 0.0, 0.0])      #km/s
[a0, e0_norm, i0, RAAN0, omega0, theta0] = rv2elorb(r0, v0, GMed)
print(a0)

136007.86020406417


In [6]:
#constants
GMed = 3.986004407799724e+5 # [km^3/sec^2]
GMe = 1
GMod = 1.32712440018e+11 #[km^3/sec^2]
GMo = GMod / GMed
GMmd = 4.9028e+3 #[km^3/sec^2]
GMm = GMmd / GMed
Red = 6378.1363 #[km]
Re = Red / a0
C20 = -4.84165371736e-4
C22 = 2.43914352398e-6
S22 = -1.40016683654e-6
theta_g = (np.pi/180)*280.4606 #[rad]
# Multiply by TU : np.sqrt((a0**3)/GMed)
nu_e = (np.pi/180)*(4.178074622024230e-3) #[rad/sec]
nu_o = (np.pi/180)*(1.1407410259335311e-5) #[rad/sec]
nu_ma = (np.pi/180)*(1.512151961904581e-4) #[rad/sec]
nu_mp = (np.pi/180)*(1.2893925235125941e-6) #[rad/sec]
nu_ms = (np.pi/180)*(6.128913003523574e-7) #[rad/sec]

nu_e = nu_e * np.sqrt((a0**3)/GMed)
nu_o = nu_o * np.sqrt((a0**3)/GMed)
nu_ma = nu_ma * np.sqrt((a0**3)/GMed)
nu_mp = nu_mp * np.sqrt((a0**3)/GMed)
nu_ms = nu_ms * np.sqrt((a0**3)/GMed)

alpha_od = 1.49619e+8 #[km]
alpha_o = alpha_od / a0

epsilon = (np.pi/180)*23.4392911 #[rad]
phi_o = (np.pi/180)*357.5256 #[rad]
Omega_plus_w = (np.pi/180)*282.94 #[rad]

In [7]:
# create heyoka variables
tau, zeta1, zeta2, zeta3, eta1, eta2, eta3, eta4 = hk.make_vars("tau", "zeta1", "zeta2", "zeta3",
                                                                "eta1", "eta2", "eta3", "eta4")

In [8]:
# Step 2: non-dimensionalise 
r0nd = r0 / a0 #[km/km] = [-] since the orbit is highly elliptical normalise with the initial semimajor axis
               # otherwise use r0nd = r0/la.norm(r0)
v0nd = v0 * math.sqrt(a0/GMed)    #[-]
t0   = 0                         #s
t0nd = t0 / math.sqrt((a0**3)/GMed) #[-]


In [9]:
# Step 3: tranform the non-dimensional ICs (r0nd, v0nd) in DROMO elements 
h0 = np.cross(r0nd, v0nd)                      # 3-components vector
e0 = - r0nd/la.norm(r0nd) - np.cross(h0, v0nd) # 3-components vector
# sigma0 from page 7. and from page 14. (initial conditions)
sigma0 = theta0 
# Initial state (sigma; tau, zeta1, zeta2, zeta3, eta1, eta2, eta3, eta4)
tau_0   = t0nd
zeta1_0 = la.norm(e0)
zeta2_0 = 0
zeta3_0 = 1/la.norm(h0)
eta1_0  = math.sin(i0/2)*math.cos((RAAN0-omega0)/2)
eta2_0  = math.sin(i0/2)*math.sin((RAAN0-omega0)/2)
eta3_0  = math.cos(i0/2)*math.sin((RAAN0+omega0)/2)
eta4_0  = math.cos(i0/2)*math.cos((RAAN0+omega0)/2)
S0 = [tau_0, zeta1_0, zeta2_0, zeta3_0, eta1_0, eta2_0, eta3_0, eta4_0]
print(S0)

[0.0, 0.9500028899051103, 0, 3.20265325797849, -0.18301270304063946, -0.18301270304063952, 0.6830127015845011, -0.683012701584501]


In [10]:
# Auxiliary relationships
s = 1 + zeta1 * hk.cos(hk.time) + zeta2 * hk.sin(hk.time)

In [11]:
# convert from Dromo to State (with heyoka variable)
def dromo2rv_hk(Lc, sigma, tau, zeta1, zeta2, zeta3, eta1, eta2, eta3, eta4):
    """
    If you want to have r,v non dimensional from dromo, let Lc be 1.
    Dromo elements to cartesian representation of state vector (page 14)
    Outputs: Dimensional components of position and velocity
    """
    s = 1 + zeta1 * hk.cos(sigma) + zeta2 * hk.sin(sigma)
    alpha = (Lc / (zeta3**2 * s))
    omegacLc = (math.sqrt(GMe/Lc**3) * Lc)

    p11 = 1-2*(eta2**2 + eta3**2)
    p12 = 2*eta1*eta2 - 2*eta4*eta3
    p21 = 2*eta1*eta2 + 2*eta4*eta3
    p22 = 1-2*(eta1**2 + eta3**2)
    p31 = 2*eta1*eta3 - 2*eta4*eta2
    p32 = 2*eta3*eta2 + 2*eta4*eta1

    x = alpha * ( p11*hk.cos(sigma) + p12*hk.sin(sigma) )
    y = alpha * ( p21*hk.cos(sigma) + p22*hk.sin(sigma) )
    z = alpha * ( p31*hk.cos(sigma) + p32*hk.sin(sigma) ) 

    V1 = -zeta3*(hk.sin(sigma)+zeta2)
    V2 =  zeta3*(hk.cos(sigma)+zeta1)

    xv = omegacLc * ( p11*V1 + p12*V2 )
    yv = omegacLc * ( p21*V1 + p22*V2 )
    zv = omegacLc * ( p31*V1 + p32*V2 ) 
    return x, y, z, xv, yv, zv 

# convert from Dromo to State (with numeric variable)
def dromo2rv(Lc, sigma, tau, zeta1, zeta2, zeta3, eta1, eta2, eta3, eta4):
    """
    If you want to have r,v non dimensional from dromo, let Lc be 1.
    Dromo elements to cartesian representation of state vector (page 14)
    Outputs: Dimensional components of position and velocity
    """
    s = 1 + zeta1 * np.cos(sigma) + zeta2 * np.sin(sigma)
    alpha = (Lc / (zeta3**2 * s))
    omegacLc = (math.sqrt(GMe/Lc**3) * Lc)

    p11 = 1-2*(eta2**2 + eta3**2)
    p12 = 2*eta1*eta2 - 2*eta4*eta3
    p21 = 2*eta1*eta2 + 2*eta4*eta3
    p22 = 1-2*(eta1**2 + eta3**2)
    p31 = 2*eta1*eta3 - 2*eta4*eta2
    p32 = 2*eta3*eta2 + 2*eta4*eta1

    x = alpha * ( p11*np.cos(sigma) + p12*np.sin(sigma) )
    y = alpha * ( p21*np.cos(sigma) + p22*np.sin(sigma) )
    z = alpha * ( p31*np.cos(sigma) + p32*np.sin(sigma) ) 

    V1 = -zeta3*(np.sin(sigma)+zeta2)
    V2 =  zeta3*(np.cos(sigma)+zeta1)

    xv = omegacLc * ( p11*V1 + p12*V2 )
    yv = omegacLc * ( p21*V1 + p22*V2 )
    zv = omegacLc * ( p31*V1 + p32*V2 ) 
    return x, y, z, xv, yv, zv 

In [13]:
X,  Y,  Z, xv, yv, zv = dromo2rv_hk(a0, hk.time, tau, zeta1, zeta2, zeta3, eta1, eta2, eta3, eta4)
X,  Y,  Z,  = X/a0, Y/a0, Z/a0
xv, yv, zv = xv*math.sqrt(a0/GMed), yv*math.sqrt(a0/GMed), zv*math.sqrt(a0/GMed)

In [14]:
############################# RHS (right hand side of the ODE) setup #############################

#Sun's position
lo = phi_o + nu_o*hk.time
lambda_o = Omega_plus_w + lo + (np.pi/180)*( (6892/3600)*hk.sin(lo) + (72/3600)*hk.sin(2*lo) )
ro = (149.619 - 2.499*hk.cos(lo) - 0.021*hk.cos(2*lo))*(10**6)

Xo = ro*hk.cos(lambda_o)
Yo = ro*hk.sin(lambda_o)*np.cos(epsilon)
Zo = ro*hk.sin(lambda_o)*np.sin(epsilon)

#Moon's position
phi_m = nu_o*hk.time
phi_ma = nu_ma*hk.time
phi_mp = nu_mp*hk.time
phi_ms = nu_ms*hk.time
L0 = phi_mp + phi_ma + (np.pi/180)*218.31617
lm = phi_ma + (np.pi/180)*134.96292
llm = phi_m + (np.pi/180)*357.5256
Fm = phi_mp + phi_ma + phi_ms + (np.pi/180)*93.27283
Dm = phi_mp + phi_ma - phi_m  + (np.pi/180)*297.85027

rm = 385000 - 20905*hk.cos(lm) - 3699*hk.cos(2*Dm - lm) - 2956*hk.cos(2*Dm) - \
     570*hk.cos(2*lm) + 246*hk.cos(2*lm - 2*Dm) - 205*hk.cos(llm - 2*Dm) - \
     171*hk.cos(lm + 2*Dm) - 152*hk.cos(lm + llm - 2*Dm)
     
lambda_m = L0 + (np.pi/180)*( (22640/3600)*hk.sin(lm) + (769/3600)*hk.sin(2*lm) - (4856/3600)*hk.sin(lm - 2*Dm) + \
     (2370/3600)*hk.sin(2*Dm) - (668/3600)*hk.sin(llm) - (412/3600)*hk.sin(2*Fm) - \
     (212/3600)*hk.sin(2*lm - 2*Dm) - (206/3600)*hk.sin(lm + llm - 2*Dm) + \
     (192/3600)*hk.sin(lm + 2*Dm) - (165/3600)*hk.sin(llm - 2*Dm) + \
     (148/3600)*hk.sin(lm - llm) - (125/3600)*hk.sin(Dm) - (110/3600)*hk.sin(lm + llm) - \
     (55/3600)*hk.sin(2*Fm - 2*Dm) )
     
βm = (np.pi/180)*( (18520/3600)*hk.sin(Fm + lambda_m - L0 + (np.pi/180)*((412/3600)*hk.sin(2*Fm) + (541/3600)*hk.sin(llm)) ) - \
     (526/3600)*hk.sin(Fm - 2*Dm) + (44/3600)*hk.sin(lm + Fm - 2*Dm) - (31/3600)*hk.sin(-lm + Fm -2*Dm) - \
     (25/3600)*hk.sin(-2*lm + Fm) - (23/3600)*hk.sin(llm + Fm - 2*Dm) + (21/3600)*hk.sin(-lm + Fm) + \
     (11/3600)*hk.sin(-llm + Fm - 2*Dm) )
     
Xm =  hk.cos(βm)*hk.cos(lambda_m)*rm
Ym = -np.sin(epsilon)*hk.sin(βm)*rm + np.cos(epsilon)*hk.cos(βm)*hk.sin(lambda_m)*rm
Zm =  np.cos(epsilon)*hk.sin(βm)*rm + hk.cos(βm)*np.sin(epsilon)*hk.sin(lambda_m)*rm

magR2 = X**2 + Y**2 + Z**2

#Earth's J2 terms
J2term1 = GMe*(Re**2)*np.sqrt(5)*C20/(2*magR2**(1./2))
J2term2 = 3/(magR2**2)
J2term3 = 15*(Z**2)/(magR2**3)
fJ2X = J2term1*X*(J2term2 - J2term3)
fJ2Y = J2term1*Y*(J2term2 - J2term3)
fJ2Z = J2term1*Z*(3*J2term2 - J2term3)

#Earth's C22 and S22 terms
x =  X*hk.cos(theta_g + nu_e*hk.time) + Y*hk.sin(theta_g + nu_e*hk.time)
y = -X*hk.sin(theta_g + nu_e*hk.time) + Y*hk.cos(theta_g + nu_e*hk.time)
z = Z
magr2 = x**2 + y**2 + z**2

C22term1 = 5*GMe*(Re**2)*np.sqrt(15)*C22/(2*magr2**(7./2))
C22term2 = GMe*(Re**2)*np.sqrt(15)*C22/(magr2**(5./2))
fC22x = C22term1*x*(y**2 - x**2) + C22term2*x
fC22y = C22term1*y*(y**2 - x**2) - C22term2*y
fC22z = C22term1*z*(y**2 - x**2)

S22term1 = 5*GMe*(Re**2)*np.sqrt(15)*S22/(magr2**(7./2))
S22term2 = GMe*(Re**2)*np.sqrt(15)*S22/(magr2**(5./2))
fS22x = -S22term1*(x**2)*y + S22term2*y
fS22y = -S22term1*x*(y**2) + S22term2*x
fS22z = -S22term1*x*y*z

fC22X = fC22x*hk.cos(theta_g + nu_e*hk.time) - fC22y*hk.sin(theta_g + nu_e*hk.time)
fC22Y = fC22x*hk.sin(theta_g + nu_e*hk.time) + fC22y*hk.cos(theta_g + nu_e*hk.time)
fC22Z = fC22z

fS22X = fS22x*hk.cos(theta_g + nu_e*hk.time) - fS22y*hk.sin(theta_g + nu_e*hk.time)
fS22Y = fS22x*hk.sin(theta_g + nu_e*hk.time) + fS22y*hk.cos(theta_g + nu_e*hk.time)
fS22Z = fS22z

#Sun's gravity
magRo2 = Xo**2 + Yo**2 + Zo**2
magRRo2 = (X - Xo)**2 + (Y - Yo)**2 + (Z - Zo)**2
fSunX = -GMo*( (X - Xo)/(magRRo2**(3./2)) + Xo/(magRo2**(3./2)) )
fSunY = -GMo*( (Y - Yo)/(magRRo2**(3./2)) + Yo/(magRo2**(3./2)) )
fSunZ = -GMo*( (Z - Zo)/(magRRo2**(3./2)) + Zo/(magRo2**(3./2)) )

#Moon's gravity 
magRm2 = Xm**2 + Ym**2 + Zm**2
magRRm2 = (X - Xm)**2 + (Y - Ym)**2 + (Z - Zm)**2
fMoonX = -GMm*( (X - Xm)/(magRRm2**(3./2)) + Xm/(magRm2**(3./2)) )
fMoonY = -GMm*( (Y - Ym)/(magRRm2**(3./2)) + Ym/(magRm2**(3./2)) )
fMoonZ = -GMm*( (Z - Zm)/(magRRm2**(3./2)) + Zm/(magRm2**(3./2)) )

############################# end of RHS setup #############################

In [15]:
# Create vectors (Non dimensional) 
r = np.array([X,  Y,  Z])  # r.shape  = (3, 1)
v = np.array([xv, yv, zv]) 
r_norm = (r[0]**2 + r[1]**2 + r[2]**2)**(1/2)  

In [16]:
# Define the unit vectors of the (local) orbital frame
io = r / r_norm
hx, hy, hz = np.cross(r, v, axis=0)
h_norm = (hx**2 + hy**2 + hz**2)**(1/2)   
ko = np.cross(r, v, axis=0) / h_norm
jo = np.cross(ko, io, axis=0)

In [18]:
# Superimpose the accelerations
apx = fJ2X + fC22X + fS22X + fSunX + fMoonX 
apy = fJ2Y + fC22Y + fS22Y + fSunY + fMoonY 
apz = fJ2Z + fC22Z + fS22Z + fSunZ + fMoonZ 
acc = np.array([apx, apy, apz])

# Project into the orbital frame (from inertial frame)
api = np.dot(io, acc) # = apx*xx + apy*xy + apz*xz
apj = np.dot(jo, acc) # = apx*yx + apy*yy + apz*yz
apk = np.dot(ko, acc) # = apx*zx + apy*zy + apz*zz

# Perturbing accelerations
a_px = api/(zeta3**4 * s**3)
a_py = apj/(zeta3**4 * s**3)
a_pz = apk/(zeta3**4 * s**3)

In [19]:

# EOMs
dtaudsigma    = 1/((zeta3**3) * (s**2)) 
dzeta1dsigma  = s * hk.sin(hk.time)   * a_px   + (zeta1 + (1+s)*hk.cos(hk.time)) * a_py 
dzeta2dsigma  = - s * hk.cos(hk.time) * a_px   + (zeta2 + (1+s)*hk.sin(hk.time)) * a_py
dzeta3dsigma  = - zeta3 * a_py
deta1dsigma   = 1/2 * a_pz * (eta4 * hk.cos(hk.time) - eta3 * hk.sin(hk.time))
deta2dsigma   = 1/2 * a_pz * (eta3 * hk.cos(hk.time) + eta4 * hk.sin(hk.time))
deta3dsigma   = 1/2 * a_pz * (-eta2* hk.cos(hk.time) + eta1 * hk.sin(hk.time))
deta4dsigma   = 1/2 * a_pz * (-eta1* hk.cos(hk.time) - eta2 * hk.sin(hk.time))
                    

In [20]:
sigma_event = []
dromo_state_event = []

# Callback for the terminal event.
def cb(ta, mr, d_sgn):
    #print("SIGMA.      when event day is reached: {}".format(ta.time))
    #print("State DROMO when event day is reached: {}".format(ta.state))  
    # Add the event time and state to the vectors:
    sigma_event.append(ta.time)
    dromo_state_event.append(ta.state)
    print(ta.state)

    return False

In [21]:
# Valore finale di sigma
sigma_fin = 55*2*np.pi     # rad  311.10986051
fin_day =  288.1276894125 # days
# numero di passi per orbita
n_orb = 50
m = 1000
N = m*n_orb
sigma_span = np.linspace(sigma0, sigma_fin, N)  
print(N)

50000


In [22]:
ev1 = hk.t_event(((tau)* math.sqrt((a0**3)/GMed)/(24*3600)) - fin_day, callback = cb)

In [23]:
print('Compiling the Taylor integrator ... (this is done only once)')
start_time = time.time()

ta = hk.taylor_adaptive(sys = [(tau,dtaudsigma),
                               (zeta1,dzeta1dsigma),
                               (zeta2,dzeta2dsigma),
                               (zeta3,dzeta3dsigma),
                               (eta1,deta1dsigma),
                               (eta2,deta2dsigma),
                               (eta3,deta3dsigma),
                               (eta4,deta4dsigma)],
                                state = S0,
                                time = sigma0,
                                tol = 1e-15,
                                compact_mode = True,
                                # The list of terminal events.
                                t_events = [ev1])

# https://bluescarni.github.io/heyoka.py/notebooks/Event%20detection.html
#end_time = time.time()
#print('Done, in');
#print("--- %s seconds ---" % (end_time - start_time))
print("\nHeyoka Taylor integrator:\n", ta)

Compiling the Taylor integrator ... (this is done only once)

Heyoka Taylor integrator:
 Taylor order            : 19
Dimension               : 8
Time                    : 0.0000000000000000
State                   : [0.0000000000000000, 0.95000288990511028, 0.0000000000000000, 3.2026532579784899, -0.18301270304063946, -0.18301270304063952, 0.68301270158450111, -0.68301270158450100]
N of terminal events    : 1



In [24]:
# Here we redefine the initial conditions since we may want to change them without recompiling the integrator
ta.time = t0
# Note the python syntax to assign directly the array elements. Here necessary
# as heyoka does not allow to change the memory location of the state
ta.state[:] =  S0

In [25]:
#start_time = time.time()
out = ta.propagate_grid(sigma_span)
end_time = time.time()

print('Done, in');
print("--- %s seconds ---" % (end_time - start_time))

#print(out)
sol = out[4] 
size1, size2 = np.shape(sol)
#print(sol[-1])
print(f'the shape of the solution (cartesian form) is {size1, size2}') 


[ 3.13344167e+02  9.47111017e-01  7.07280383e-02  3.20227596e+00
 -1.99836626e-01 -1.64631234e-01  6.87701298e-01 -6.78254233e-01]
Done, in
--- 7.602319002151489 seconds ---
the shape of the solution (cartesian form) is (45886, 8)


In [27]:
df_dromo = pd.DataFrame(sol, columns=["tau", "z1", "z2", "z3", "h1", "h2", "h3", "h4"])
#print(df_dromo.tail(1))


#print(sigma_event)
#print(dromo_state_event)

#print(sol[-1])

rv_event = dromo2rv(a0, sigma_event[0], *dromo_state_event[0])
print(rv_event)

r_norma = (rv_event[0]**2+rv_event[1]**2+rv_event[2]**2)**0.5
print("norm r: ", r_norma)
#print("r error:", (np.abs(r_norma-263418.3747296585))*1000)
v_norma = (rv_event[3]**2+rv_event[4]**2+rv_event[5]**2)**0.5
print("norm v: ", v_norma)
#print("v error:", (np.abs(v_norma-0.35241893529632706))*1000)


(19145.016975618517, 180343.87951514113, 104755.46149450363, -0.0004221842692871306, 0.001231985812168634, 0.0006827096384354219)
norm r:  209437.70736544768
norm v:  0.001470415264145113
